In [1]:
import pandas as pd
academic = pd.read_feather('./feather/academic.feather')#.to_csv('./tables/academics.csv',index=False)

# Tables concerning inproceedings (conference articles)
improceeding = pd.read_feather('./feather/improceeding_table.feather')#.to_csv('./tables/improceedings=.csv',index=False)
author_inproceeding = pd.read_feather('./feather/author_inproceedings.feather')#.to_csv('./tables/inproceedings_authors.csv',index=False)
booktitle_improceeding = pd.read_feather('./feather/improceeding_booktitle.feather')#.to_csv('./tables/improceedings_booktitles.csv',index=False)

# All tables concerning articles(journal articles)
article = pd.read_feather('./feather/articles_table.feather')#.to_csv('./tables/articles.csv',index=False)
author_article = pd.read_feather('./feather/author_article.feather')#.to_csv('./tables/article_authors.csv',index=False)
journal = pd.read_feather('./feather/journals_table.feather')#.to_csv('./tables/journals.csv',index=False)


# All tables concerning proceedings (conference proceedings => bundled conference articles)
editor_proceeding = pd.read_feather('./feather/editor_proceedings.feather')#.to_csv('./tables/proceedings_editors.csv',index=False)
publisher_proceeding = pd.read_feather('./feather/publisher_proceeding.feather')#.to_csv('./tables/proceedings_publishers.csv',index=False)
publisher = pd.read_feather('./feather/publisher_table.feather')#.to_csv('./tables/publishers.csv',index=False)
proceeding = pd.read_feather('./feather/proceeding_table.feather')#.to_csv('./tables/proceedings.csv',index=False)
booktitle_proceeding = pd.read_feather('./feather/proceeding_booktitle.feather')#.to_csv('./tables/proceedings_booktitles.csv',index=False)

# All tables concerning booktiles (conferences)
booktitle = pd.read_feather('./feather/booktitles_table.feather')#.to_csv('./tables/booktitles.csv',index=False)

In [2]:
article

,key,title,journal_id
0,conf/www/RamamohanaraoF07,Patterns Based Classifiers.,1936
1,conf/www/ZhaiL07,Extracting Web Data Using Instance-Based Learn...,1936
2,conf/www/NguKN07,Editorial.,1936
3,conf/www/Ericsson07,The Effects of XML Compression on SOAP Perform...,1936
4,conf/www/MartinBMMPSMSS07,Bringing Semantics to Web Services with OWL-S.,1936
...,...,...,...
2987941,tr/gte/TR-0244-12-93-165,MetaObject Protocol Concepts for a RISC Object...,545
2987942,tr/gte/TR-0146-06-91-165,Towards a Transaction Management System for DOM.,545
2987943,tr/gte/TR-0222-10-92-165,DARWIN: On the Incremental Migration of Legacy...,545
2987944,tr/dec/SRC1997-018,"The 1995 SQL Reunion: People, Project, and Pol...",395


In [3]:
# remove duplicated rows from all tables if they exist
improceeding = improceeding.drop_duplicates(keep='first')
author_inproceeding = author_inproceeding.drop_duplicates(keep='first')
booktitle_improceeding = booktitle_improceeding.drop_duplicates(keep='first')
article = article.drop_duplicates(keep='first')
author_article = author_article.drop_duplicates(keep='first')
journal = journal.drop_duplicates(keep='first')
editor_proceeding = editor_proceeding.drop_duplicates(keep='first')
publisher_proceeding = publisher_proceeding.drop_duplicates(keep='first')
publisher = publisher.drop_duplicates(keep='first')
proceeding = proceeding.drop_duplicates(keep='first')
booktitle_proceeding = booktitle_proceeding.drop_duplicates(keep='first')
booktitle = booktitle.drop_duplicates(keep='first')

#rename columns of tables
academic.columns = ['academic_id','academic_name']
article.columns = ['article_id','article_title','journal_id']
author_article.columns = ['academic_id','article_id']
journal.columns = ['journal_id','journal_name']
proceeding.columns = ['proceeding_id','proceeding_title','year','volume']
publisher.columns = ['publisher_id','publisher_name']
booktitle.columns = ['booktitle_id','booktitle_name']
booktitle_proceeding.columns = ['proceeding_id','booktitle_id']
improceeding.columns = ['improceeding_id','improceeding_title','pages','year']
booktitle_improceeding.columns = ['improceeding_id','booktitle_id']
publisher_proceeding.columns = ['publisher_id','proceeding_id']
editor_proceeding.columns = ['academic_id','proceeding_id']
author_inproceeding.columns = ['academic_id','improceeding_id']

In [4]:
# create the duckdb database in memory
import duckdb
conn = duckdb.connect()
cursor = conn.cursor()

# create tables

# academic table
cursor.execute('CREATE TABLE academic AS SELECT * FROM academic')

# Create the 'improceeding' table
cursor.execute('CREATE TABLE improceeding AS SELECT * FROM improceeding')

# Create the 'author_inproceeding' table
cursor.execute('CREATE TABLE author_inproceeding AS SELECT * FROM author_inproceeding')


# Create the 'booktitle_improceeding' table
cursor.execute('CREATE TABLE booktitle_improceeding AS SELECT *  FROM booktitle_improceeding')


# Create the 'article' table
cursor.execute('CREATE TABLE article AS SELECT * FROM article')

# Create the 'author_article' table
cursor.execute('CREATE TABLE author_article AS SELECT * FROM author_article')

# Create the 'journal' table
cursor.execute('CREATE TABLE journal AS SELECT * FROM journal')


# Create the 'editor_proceeding' table
cursor.execute('CREATE TABLE editor_proceeding AS SELECT * FROM editor_proceeding')


# Create the 'publisher_proceeding' table
cursor.execute('CREATE TABLE publisher_proceeding AS SELECT * FROM publisher_proceeding')


# Create the 'publisher' table
cursor.execute('CREATE TABLE publisher AS SELECT * FROM publisher')

# Create the 'proceeding' table
cursor.execute('CREATE TABLE proceeding AS SELECT * FROM proceeding')

# Create the 'booktitle_proceeding' table
cursor.execute('CREATE TABLE booktitle_proceeding AS SELECT * FROM booktitle_proceeding')


# Create the 'booktitle' table
cursor.execute('CREATE TABLE booktitle AS SELECT * FROM booktitle')

In [5]:
import duckdb

# Create the index
cursor = conn.cursor()
cursor.execute("CREATE INDEX academic_article_index ON author_article (academic_id, article_id)")
cursor.execute("CREATE INDEX booktitle_improceeding_idx ON booktitle_improceeding (booktitle_id)")
cursor.execute("CREATE INDEX improceeding_idx ON improceeding (improceeding_id)")

In [6]:
#Booktitle_name
queryE1 = """
            SELECT publisher.publisher_name 
            FROM publisher 
            JOIN publisher_proceeding ON publisher.publisher_id=publisher_proceeding.publisher_id 
            JOIN proceeding ON proceeding.proceeding_id=publisher_proceeding.proceeding_id 
            WHERE proceeding.proceeding_title LIKE '%PODS%' AND publisher.publisher_name IS NOT NULL
            GROUP BY publisher_name
            """

E1 = cursor.execute(queryE1).fetchdf()
E1

,publisher_name
0,ACM
1,CWI


In [7]:
queryE2_1 = """
SELECT article.article_title 
FROM article JOIN author_article ON author_article.article_id=article.article_id 
JOIN academic ON academic.academic_id=author_article.academic_id 
JOIN journal ON article.journal_id=journal.journal_id 
WHERE journal.journal_name='Theory Comput. Syst.' 
AND academic.academic_name='Martin Grohe' 
ORDER BY article.article_title"""
queryE2_2 = """
SELECT article.article_title 
FROM article, journal, author_article, academic
WHERE article.journal_id=journal.journal_id 
AND article.article_id=author_article.article_id 
AND academic.academic_name='Martin Grohe' 
AND journal_name='Theory Comput. Syst.'"""

E2 = cursor.execute(queryE2_1).fetchdf()
E2

,article_title
0,Database Query Processing Using Finite Cursor ...
1,Learnability and Definability in Trees and Sim...
2,Tight Lower and Upper Bounds for the Complexit...


In [8]:
queryM1 = """
SELECT COUNT(year) as number_of_articles_published, year 
FROM booktitle 
JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id 
JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id 
WHERE lower(booktitle.booktitle_name) LIKE '%sigmod%' 
AND year='2022' 
GROUP BY year"""

M1 = cursor.execute(queryM1).fetchdf()
M1

,number_of_articles_published,year
0,282,2022


In [121]:
queryM2 = """Select * FROM proceeding ORDER BY proceeding.year"""
duckdb.query(queryM2).to_df()

,proceeding_id,proceeding_title,year,volume
0,conf/aieeire/1951,Papers and discussions presented at the 1951 J...,1951,NaN
1,conf/acm/1952t,Proceedings of the 1952 ACM national meeting (...,1952,NaN
2,conf/acm/1952p,Proceedings of the 1952 ACM national meeting (...,1952,NaN
3,conf/aieeire/1953e,Papers and discussions presented at the 1953 e...,1953,NaN
4,conf/aieeire/1953w,Proceedings of the 1953 western computer confe...,1953,NaN
...,...,...,...,...
53398,conf/intellisys/2022-1,Intelligent Systems and Applications - Proceed...,2023,542
53399,conf/intellisys/2022-2,Intelligent Systems and Applications - Proceed...,2023,543
53400,conf/intellisys/2022-3,Intelligent Systems and Applications - Proceed...,2023,544
53401,conf/dps/2022,Intelligent and Safe Computer Systems in Contr...,2023,545


In [9]:
queryM3 ="""
            SELECT MEDIAN(Frequency) FROM (
            SELECT count(improceeding.improceeding_id) as frequency,improceeding.year AS improceedings_per_year
            FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id 
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id 
            WHERE booktitle.booktitle_name='CIDR'
            GROUP BY improceeding.year
            ORDER BY improceeding.year ASC
            )
            """

queryM3_1 = """
            SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY frequency) as median_frequency
            FROM (
            SELECT count(improceeding.improceeding_id) as frequency, improceeding.year AS improceedings_per_year
            FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id = booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id = booktitle_improceeding.improceeding_id
            WHERE booktitle.booktitle_name = 'CIDR'
            GROUP BY improceeding.year
            ORDER BY improceeding.year ASC
            )
            """

M3 = cursor.execute(queryM3_1).fetchdf()
M3

,median_frequency
0,45.0


In [10]:
queryM4_1 = "SELECT COUNT(year), year FROM booktitle JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id WHERE booktitle.booktitle_name='SIGMOD' GROUP BY year ORDER BY year"

queryM4_2 = """
        SELECT * 
            FROM 
                (SELECT COUNT(year) as improceeding_frequency_10, year 
                FROM 
                    (SELECT improceeding_id 
                    FROM 
                        (SELECT COUNT(improceeding.improceeding_id) as improceeding_authors, improceeding.improceeding_id 
                        FROM author_inproceeding 
                        JOIN improceeding ON improceeding.improceeding_id=author_inproceeding.improceeding_id 
                        GROUP BY improceeding.improceeding_id 
                        ORDER BY improceeding_authors) x 
                        WHERE x.improceeding_authors > 10) y 
                        JOIN improceeding ON improceeding.improceeding_id=y.improceeding_id 
                        JOIN booktitle_improceeding ON booktitle_improceeding.improceeding_id=improceeding.improceeding_id 
                        JOIN booktitle ON booktitle.booktitle_id=booktitle_improceeding.booktitle_id 
                        WHERE booktitle.booktitle_name LIKE '%SIGMOD%' 
                        GROUP BY year 
                        ORDER BY improceeding_frequency_10 DESC)"""
 
queryM4 = """
            SELECT MAX(improceeding_frequency_10) as number_of_SIGMOD_conference_higher_than_10_authors, ANY_VALUE(year) as year 
            FROM 
                (SELECT COUNT(year) as improceeding_frequency_10, year 
                FROM 
                    (SELECT improceeding_id 
                    FROM 
                        (SELECT COUNT(improceeding.improceeding_id) as improceeding_authors, improceeding.improceeding_id 
                        FROM author_inproceeding 
                        JOIN improceeding ON improceeding.improceeding_id=author_inproceeding.improceeding_id 
                        GROUP BY improceeding.improceeding_id 
                        ORDER BY improceeding_authors) x 
                        WHERE x.improceeding_authors > 10) y 
                        JOIN improceeding ON improceeding.improceeding_id=y.improceeding_id 
                        JOIN booktitle_improceeding ON booktitle_improceeding.improceeding_id=improceeding.improceeding_id 
                        JOIN booktitle ON booktitle.booktitle_id=booktitle_improceeding.booktitle_id 
                        WHERE booktitle.booktitle_name LIKE '%SIGMOD%' 
                        GROUP BY year 
                        ORDER BY improceeding_frequency_10 DESC)"""

queryM4_3 = """
            SELECT MAX(improceeding_frequency_10) as number_of_SIGMOD_conference_higher_than_10_authors, ANY_VALUE(year) as year
            FROM (
            SELECT COUNT(year) as improceeding_frequency_10, year
            FROM improceeding
            JOIN booktitle_improceeding ON booktitle_improceeding.improceeding_id = improceeding.improceeding_id
            JOIN booktitle ON booktitle.booktitle_id = booktitle_improceeding.booktitle_id
            WHERE booktitle.booktitle_name LIKE '%SIGMOD%'
            GROUP BY year
            HAVING COUNT(*) > 10
            ORDER BY improceeding_frequency_10 DESC
            )
            """

M4 = cursor.execute(queryM4_3).fetchdf()
M4

,number_of_SIGMOD_conference_higher_than_10_authors,year
0,299,2021


In [11]:
#queryM5_1 = """SELECT COUNT(academic.academic_name) AS author_frequency, academic.academic_name FROM academic JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id WHERE proceeding.proceeding_title LIKE '%PODS%' GROUP BY academic.academic_name ORDER BY author_frequency DESC"

#Duplicates proceedings 

queryM5_0 = """
            SELECT * 
            FROM academic
            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id
            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id
            WHERE academic.academic_name LIKE '%Josep Llu%'
            """

queryM5_1 = """
            SELECT COUNT(academic.academic_id) AS author_frequency, ANY_VALUE(academic.academic_name)
            FROM academic 
            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
            WHERE booktitle.booktitle_name='PODS' 
            GROUP BY academic.academic_id
            ORDER BY author_frequency DESC"""


queryM5 = """
            SELECT academic_name, author_frequency FROM
            (SELECT COUNT(academic.academic_name) AS author_frequency, academic.academic_name 
            FROM academic 
            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
            WHERE booktitle.booktitle_name LIKE '%PODS%' 
            GROUP BY academic.academic_name)
            WHERE author_frequency = (SELECT MAX(author_frequency) FROM     (SELECT COUNT(academic.academic_name) AS author_frequency, academic.academic_name 
                                                                            FROM academic 
                                                                            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
                                                                            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
                                                                            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
                                                                            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
                                                                            WHERE booktitle.booktitle_name LIKE '%PODS%' 
                                                                            GROUP BY academic.academic_name))"""

M5 = cursor.execute(queryM5).fetchdf()
M5

,academic_name,author_frequency
0,Maurizio Lenzerini,3
1,Leonid Libkin,3
2,Josep Lluís Larriba-Pey,3


In [12]:
#Find the academic with the most publishings, and give the academic_id back (together with the amount of publishings)
queryM6_1 = """
            SELECT MAX(Z.total_publishings), ANY_VALUE(Z.academic_id)
            FROM
            (SELECT X.academic_id, number_of_improceedings, number_of_journal_articles, number_of_improceedings+number_of_journal_articles as total_publishings
            FROM 
                (SELECT COUNT(author_inproceeding.academic_id) as number_of_improceedings, author_inproceeding.academic_id 
                FROM academic 
                JOIN author_inproceeding ON author_inproceeding.academic_id=academic.academic_id 
                GROUP BY author_inproceeding.academic_id) X 
                JOIN 
                    (SELECT COUNT(author_article.academic_id) as number_of_journal_articles, author_article.academic_id 
                    FROM academic 
                    JOIN author_article ON author_article.academic_id=academic.academic_id 
                    GROUP BY author_article.academic_id) Y 
                ON X.academic_id=Y.academic_id ORDER BY total_publishings DESC) Z
            WHERE NOT(Z.academic_id=-1)"""

#Giving a academic_id (here hardcoded as '968212') give there number of different conferences published (in this case 90)
queryM6_2 = """
            SELECT COUNT(booktitle_name) FROM
            (SELECT COUNT(booktitle.booktitle_name), booktitle.booktitle_name FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id
            JOIN author_inproceeding ON author_inproceeding.improceeding_id=improceeding.improceeding_id
            JOIN academic ON academic.academic_id=author_inproceeding.academic_id
            WHERE academic.academic_id=968212
            GROUP BY booktitle.booktitle_name)"""

#Combine both subqueries from above
queryM6 = """
            SELECT COUNT(booktitle_name) AS number_of_different_conferences_published FROM
            (SELECT COUNT(booktitle.booktitle_name), booktitle.booktitle_name FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id
            JOIN author_inproceeding ON author_inproceeding.improceeding_id=improceeding.improceeding_id
            JOIN academic ON academic.academic_id=author_inproceeding.academic_id
            WHERE academic.academic_id=(SELECT A.academic_id
                                        FROM
                                        (SELECT MAX(Z.total_publishings), ANY_VALUE(Z.academic_id) AS academic_id
                                        FROM
                                        (SELECT X.academic_id, number_of_improceedings, number_of_journal_articles, number_of_improceedings+number_of_journal_articles as total_publishings
                                        FROM 
                                            (SELECT COUNT(author_inproceeding.academic_id) as number_of_improceedings, author_inproceeding.academic_id 
                                            FROM academic 
                                            JOIN author_inproceeding ON author_inproceeding.academic_id=academic.academic_id 
                                            GROUP BY author_inproceeding.academic_id) X 
                                            JOIN 
                                                (SELECT COUNT(author_article.academic_id) as number_of_journal_articles, author_article.academic_id 
                                                FROM academic 
                                                JOIN author_article ON author_article.academic_id=academic.academic_id 
                                                GROUP BY author_article.academic_id) Y 
                                            ON X.academic_id=Y.academic_id ORDER BY total_publishings DESC) Z
                                        WHERE NOT(Z.academic_id=-1)) A)
            GROUP BY booktitle.booktitle_name)
            """

M6 = cursor.execute(queryM6).fetchdf()
M6

,number_of_different_conferences_published
0,90


In [13]:
queryH1_help1 = "CREATE INDEX index_name ON author_article (academic_id, article_id);"

queryH1_1 = """
        SELECT academic_id, author_inproceeding.improceeding_id, improceeding.improceeding_title
        FROM booktitle 
        JOIN booktitle_improceeding ON booktitle.booktitle_id=booktitle_improceeding.booktitle_id 
        JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id 
        JOIN author_inproceeding ON author_inproceeding.improceeding_id=improceeding.improceeding_id 
        WHERE booktitle_name LIKE '%ICDT%' AND year='2020'"""

queryH1_2 = """
                SELECT a2.academic_name, COUNT(*) as coauthor_count
                FROM author_article aa1
                JOIN author_article aa2 ON aa1.article_id = aa2.article_id
                JOIN academic a1 ON aa1.academic_id = a1.academic_id
                JOIN academic a2 ON aa2.academic_id = a2.academic_id
                WHERE a1.academic_id = '168668' AND a2.academic_id != '168668'
                GROUP BY a2.academic_name
                ORDER BY coauthor_count DESC
                LIMIT 1
                """

queryH1_3 = """
                SELECT ANY_VALUE(a1.academic_id), ANY_VALUE(a1.academic_name), ANY_VALUE(a2.academic_name) as coauthor, COUNT(*) as coauthor_count
                FROM author_article aa1
                JOIN author_article aa2 ON aa1.article_id = aa2.article_id
                JOIN academic a1 ON aa1.academic_id = a1.academic_id
                JOIN academic a2 ON aa2.academic_id = a2.academic_id
                WHERE a2.academic_id != a1.academic_id
                GROUP BY a1.academic_id, a2.academic_name
                ORDER BY a1.academic_id, coauthor_count DESC
                LIMIT 5
            """

H1 = cursor.execute(queryH1_3).fetchdf()
H1

Error: Invalid Error: bad allocation

In [15]:
queryH2 = """
            SELECT 
            CASE 
                WHEN a1.academic_id IN (SELECT academic_id 
                                        FROM author_inproceeding 
                                        WHERE improceeding_id IN 
                                            (SELECT improceeding_id 
                                            FROM author_inproceeding 
                                            WHERE academic_id = a2.academic_id)) 
                                        THEN 0
                ELSE 1
            END AS distance
            FROM academic a1
            JOIN academic a2 ON a1.academic_id <> a2.academic_id
            WHERE a1.academic_name = 'Maurizio Lenzerini' AND a2.academic_name = 'Martin Grohe'
            """

queryH2_1 = """
WITH RECURSIVE co_publishers AS (
  SELECT a1.academic_id, a2.academic_id AS co_publisher_id, 0 AS distance
  FROM academic a1
  JOIN author_inproceeding ai1 ON a1.academic_id = ai1.academic_id
  JOIN author_inproceeding ai2 ON ai1.improceeding_id = ai2.improceeding_id AND ai2.academic_id <> a1.academic_id
  JOIN academic a2 ON ai2.academic_id = a2.academic_id
  WHERE a1.academic_name = 'Maurizio Lenzerini'
  UNION ALL
  SELECT cp.academic_id, a.academic_id AS co_publisher_id, cp.distance + 1 AS distance
  FROM co_publishers cp
  JOIN author_inproceeding ai1 ON cp.co_publisher_id = ai1.academic_id
  JOIN author_inproceeding ai2 ON ai1.improceeding_id = ai2.improceeding_id AND ai2.academic_id <> cp.co_publisher_id
  JOIN academic a ON ai2.academic_id = a.academic_id
)
SELECT MIN(distance) AS distance
FROM co_publishers
WHERE co_publisher_id IN (SELECT academic_id FROM academic WHERE academic_name = 'Martin Grohe')
            """

H2 = cursor.execute(queryH2_1).fetchdf()
H2

Error: Invalid Error: bad allocation

In [16]:
# The query returns the names of all pairs of academics who have edited the same proceedings, 
# but excludes pairs where the academics are the same.

queryB1 = """
        SELECT a1.academic_name, a2.academic_name
        FROM academic a1
        JOIN editor_proceeding ep1 ON a1.academic_id = ep1.academic_id
        JOIN proceeding p ON ep1.proceeding_id = p.proceeding_id
        JOIN editor_proceeding ep2 ON p.proceeding_id = ep2.proceeding_id
        JOIN academic a2 ON ep2.academic_id = a2.academic_id
        WHERE a1.academic_id <> a2.academic_id
        """

duckdb.query(queryB1).to_df()

# B1 = cursor.execute(queryB1).fetchdf()
# B1

,academic_name,academic_name_2
0,Ivan Jureta,Renata S. S. Guizzardi
1,George Lepouras,Alan J. Dix
2,Shaoyi He,Lean Yu
3,Pierre Maret,Laurent Vercouter
4,Stefan Trausan-Matu,Kristy Elizabeth Boyer
...,...,...
434273,Slimane Hammoudi,Manuel I. Capel Tuñón
434274,Lotfi A. Zadeh,Yingxu Wang 0002
434275,Lotfi A. Zadeh,Youakim Badr
434276,Slimane Hammoudi,Markus Helfert


In [19]:
queryB2 = """
            """


,academic_name,booktitle_name,improceeding_title,proceeding_title
0,Alfredo Cuzzocrea,DOLAP,Towards Local Post-hoc Recommender Systems Exp...,Proceedings of the sixteenth international wor...
1,Johann Eder,ADBIS (Short Papers),OCEL: A Standard for Object-Centric Event Logs.,"Advances in Databases and Information Systems,..."
2,Ladjel Bellatreche,DOLAP,Towards Local Post-hoc Recommender Systems Exp...,Proceedings of the sixteenth international wor...
3,Kostas Stefanidis,DOLAP,Towards Local Post-hoc Recommender Systems Exp...,Proceedings of the 24th International Workshop...
4,Pavel Zezula,SISAP,Similarity vs. Relevance: From Simple Searches...,Similarity Search and Applications - 6th Inter...
...,...,...,...,...
9393971,Theresa Ahearn,SMC,Multi-task based Image Aesthetics Quality Eval...,Driving Scientific and Engineering Discoveries...
9393972,Becky Verastegui,SMC,Multi-task based Image Aesthetics Quality Eval...,Driving Scientific and Engineering Discoveries...
9393973,Arthur Barney Maccabe,SMC,A Dataset and System for Real-Time Gun Detecti...,Driving Scientific and Engineering Discoveries...
9393974,Theresa Ahearn,SMC,A Dataset and System for Real-Time Gun Detecti...,Driving Scientific and Engineering Discoveries...
